In [1]:
from unityagents import UnityEnvironment
import numpy as np

import gym

import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
env = UnityEnvironment("Banana.app")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
class Qnetwork(nn.Module):
    def __init__(self,state_size,action_size):
        # initialising the super class properties
        super(Qnetwork,self).__init__()
        
        # defining layers
        self.fc1 = nn.Linear(state_size,32)
        self.fc2 = nn.Linear(32,64)
        self.fc3 = nn.Linear(64,128)
        self.out = nn.Linear(128,action_size)
        
    def forward(self,states):
        x = F.relu(self.fc1(states))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        out = F.relu(self.out(x))
        return out

In [4]:
class ReplayBuffer:
    def __init__(self,buffer_size,batch_size):
        self.buffer = deque(maxlen=buffer_size)
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.batch_size = batch_size
        
    def add(self,state,action,reward,next_state,done):
        e = self.experience(state,action,reward,next_state,done)
        self.buffer.append(e)
    
    def sample(self,random=True):
        if random:
            choose = np.random.choice(range(len(self.buffer)),self.batch_size,replace=False)
            return [self.buffer[i] for i in choose]
        else:
            pass
        
    def __len__(self):
        return len(self.buffer)

In [5]:
class Agent:
    def __init__(self,state_size,action_size,gamma=1.0,alpha=1e-3,lr=1e-3,
                     buffer_size=1e4,batch_size=64):
        # defining local and target networks
        self.qnet_local = Qnetwork(state_size,action_size)
        self.qnet_target = Qnetwork(state_size,action_size)
        
        # experience replay buffer
        self.memory = ReplayBuffer(buffer_size,batch_size)
        
        # defining variables
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.gamma = gamma
        self.alpha = alpha
        self.lr = lr
        
        # optimizer
        self.optimizer = optim.Adam(self.qnet_local.parameters(),lr=self.lr)
    
    def step(self,state,action,reward,next_state,done):
        # add sample to the memory buffer
        self.memory.add(state,action,reward,next_state,done)
        
        # use replay buffer to learn if it has enough samples
        if len(self.memory) > self.batch_size:
            experiences = self.memory.sample()
            self.learn(experiences)
    
    def learn(self,experiences):
        states, actions, rewards, next_states, dones = experiences
        
        # predicted action value
        q_pred = self.qnet_local(states)
        
        # target action value
        q_target = rewards + self.gamma*(1-dones)*self.qnet_target(next_states)[actions]
        
        # defining loss
        loss = F.mse_loss(q_pred,q_target)
        
        # running backprop and optimizer step
        loss.backward()
        self.optimizer.step()
        
        
    def act(self,state):
        action = self.model.forward(state)
        return action

In [4]:
def dqn_train(max_episodes):
    
        for eps in range(max_episodes):
            
            # reset env state
            env_info = env.reset(train_mode=True)[brain_name]
            state = env_info.vector_observation[0]
            
            done = env_info.local_done[0]
            
            while not done:
                # choose action using the local q-network
                action = agent.act(state)

                # taking action in the env
                env_info = env.step(vector_action=action)[brain_name]

                # getting next_state,reward,done from the env
                next_state = env_info.vector_observation[0]
                reward = env_info.rewards[0]
                done = env_info.local_done[0]
                
                # using agent to perform a learning step and save the sample in the memory
                agent.step(state,action,reward,next_state,done)

SyntaxError: invalid syntax (<ipython-input-4-59d8f3fad3b4>, line 16)

{'BananaBrain': <unityagents.brain.BrainParameters at 0x10cfe9e80>}